In [247]:
import pandas as pd

airports = pd.read_csv('data/airports.csv')

In [248]:
airportLocations = {}
for index, row in airports.iterrows():
    airportLocations[row["IATA_CODE"]] = (row["LATITUDE"], row["LONGITUDE"])

In [316]:
import cPickle
import datetime
import matplotlib.pyplot as plt
import mplleaflet

origin = 'ORD'
destination = 'LAX'
direct = False

if direct:
    policyFile = 'Results/Winter/Direct/' + origin + 'to' + destination + '_direct_policy.pkl'
    valueFile = 'Results/Winter/Direct/' + origin + 'to' + destination + '_direct_value.pkl'
else:
    policyFile = 'Results/Winter/Connected/' + origin + 'to' + destination + '_policy.pkl'
    valueFile = 'Results/Winter/Connected/' + origin + 'to' + destination + '_value.pkl'

print 'loading optimal policy pkl'
optimal_policy = {}
with open(policyFile, "rb") as input_file:
    optimal_policy = cPickle.load(input_file)
print 'done loading optimal policy'

print 'loading value function pkl'
value_function = {}
with open(valueFile, "rb") as input_file:
    value_function = cPickle.load(input_file)
print 'done loading value function'

state = (origin, datetime.datetime(2015, 1, 11, 8, 30), 0)
print 'Value at start state is ', value_function[state]
path = [(state, None)]
actions = []

while True:
    print '\n'
    print 's: ', state, value_function[state]
    if state[0] == destination:
        break
    else:
        action = optimal_policy[state]
        print 'a: ', action
        actions.append(action[1])
        state = (action[2],action[3],state[2]+1)
        path.append((state,action,state[2]+1))

airportsVisited = [i[0][0] for i in path]
print airportsVisited

flightsToTake = [(i[1][0],i[1][4]) for i in path[1:]]
print flightsToTake

points = []
for airport in airportsVisited:
    points.append(airportLocations[airport])

lats = [i[0] for i in points]
lngs = [i[1] for i in points]

if direct:
    plt.plot(lngs, lats,'ob-')
else:
    plt.plot(lngs, lats,'or-')

# plt.show()
for i, label in enumerate(points): 
   plt.text (lngs[i], lats[i], airportsVisited[i])

mplleaflet.show()

loading optimal policy pkl
done loading optimal policy
loading value function pkl
done loading value function
Value at start state is  -594.487674586


s:  ('ORD', datetime.datetime(2015, 1, 11, 8, 30), 0) -594.487674586
a:  ('US602', datetime.datetime(2015, 1, 11, 11, 5), 'PHX', datetime.datetime(2015, 1, 11, 13, 38), 216.0)


s:  ('PHX', datetime.datetime(2015, 1, 11, 13, 38), 1) -235.273303128
a:  ('WN336', datetime.datetime(2015, 1, 11, 14, 10), 'LAS', datetime.datetime(2015, 1, 11, 15, 10), 65.0)


s:  ('LAS', datetime.datetime(2015, 1, 11, 15, 10), 2) -105.23818809
a:  ('VX1493', datetime.datetime(2015, 1, 11, 15, 33), 'LAX', datetime.datetime(2015, 1, 11, 17, 51), 73.0)


s:  ('LAX', datetime.datetime(2015, 1, 11, 17, 51), 3) 0
['ORD', 'PHX', 'LAS', 'LAX']
[('US602', 216.0), ('WN336', 65.0), ('VX1493', 73.0)]
